<a href="https://colab.research.google.com/github/vortexash/Machine-Learning/blob/master/poptagger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from nltk import word_tokenize ,pos_tag

In [20]:
print (pos_tag(word_tokenize("I'm learning NLP")))

[('I', 'PRP'), ("'m", 'VBP'), ('learning', 'VBG'), ('NLP', 'NNP')]


In [17]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

True

In [24]:
 #Picking a corpus to train the pos tagger
  tagged_sentence = nltk.corpus.treebank.tagged_sents()
  print(tagged_sentence[0])
  print("Tagged sentence:",len(tagged_sentence))
  print("Tagged words:",len(nltk.corpus.treebank.tagged_words()))

[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')]
Tagged sentence: 3914
Tagged words: 100676


In [0]:
 ### traning our Pos tagger using sklearn.
  def features(sentence,index):
    """ sentence:[w1,w2, ...], index: the index of the word """
    return{
        'word': sentence[index],
        'is_first':index == 0,
        'is_last' : index == len(sentence)-1,
        'is_capitalized' : sentence[index][0].upper() == sentence[index][0],
        'is_all_caps': sentence[index].upper() == sentence[index],
        'is_all_lower': sentence[index].lower() == sentence[index],
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
        'has_hyphen': '-' in sentence[index],
        'is_numeric': sentence[index].isdigit(),
        'capitals_inside': sentence[index][1:].lower() != sentence[index][1:]
    }

In [29]:
import pprint
pprint.pprint(features(['This','is','a','sentence'],2))
{'capitals_inside': False,
 'has_hyphen': False,
 'is_all_caps': False,
 'is_all_lower': True,
 'is_capitalized': False,
 'is_first': False,
 'is_last': False,
 'is_numeric': False,
 'next_word': 'sentence',
 'prefix-1': 'a',
 'prefix-2': 'a',
 'prefix-3': 'a',
 'prev_word': 'is',
 'suffix-1': 'a',
 'suffix-2': 'a',
 'suffix-3': 'a',
 'word': 'a'}

{'capitals_inside': False,
 'has_hyphen': False,
 'is_all_caps': False,
 'is_all_lower': True,
 'is_capitalized': False,
 'is_first': False,
 'is_last': False,
 'is_numeric': False,
 'next_word': 'sentence',
 'prefix-1': 'a',
 'prefix-2': 'a',
 'prefix-3': 'a',
 'prev_word': 'is',
 'suffix-1': 'a',
 'suffix-2': 'a',
 'suffix-3': 'a',
 'word': 'a'}


{'capitals_inside': False,
 'has_hyphen': False,
 'is_all_caps': False,
 'is_all_lower': True,
 'is_capitalized': False,
 'is_first': False,
 'is_last': False,
 'is_numeric': False,
 'next_word': 'sentence',
 'prefix-1': 'a',
 'prefix-2': 'a',
 'prefix-3': 'a',
 'prev_word': 'is',
 'suffix-1': 'a',
 'suffix-2': 'a',
 'suffix-3': 'a',
 'word': 'a'}

In [0]:
## small helper function to strip the tags from tagged copus and feed it to our classifier:
 def untag(tagged_sentence):
    return[w for w, t in tagged_sentence]

In [39]:
## Split the dataset for traning and testing
cutoff =int(.75 * len(tagged_sentence))
traning_sentences = tagged_sentence[:cutoff]
test_sentences = tagged_sentence[cutoff:]

print(len(traning_sentences))
print(len(test_sentences))

def transform_to_dataset(tagged_sentence):
  X,y=[] ,[]
  for tagged in tagged_sentence:
      for index in range(len(tagged)):
        X.append(features(untag(tagged),index))
        y.append(tagged[index][1])
  return X,y
X,y = transform_to_dataset(traninig_sentences)

2935
979


In [43]:
### traning model
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline

clf= Pipeline([
     ('vectorizer', DictVectorizer(sparse=False)),
    ('classifier', DecisionTreeClassifier(criterion='entropy'))
])
clf.fit(X[:10000],y[:10000]) ## using only the first 10K sample
 
print("Traning Complete")
X_test,y_test = transform_to_dataset(test_sentences)
print("Accuracy:",clf.score(X_test,y_test))
  

Traning Complete
Accuracy: 0.8959906797364615


In [0]:
### tagging 
def pos_tag(sentence):
  tags = clf.predict([features(sentence,index) for index in range(len(sentence))])
  return zip(sentence,tags)
jk=(pos_tag(word_tokenize("This is my friend,Jimmy.")))

In [61]:
print(jk)